# Código Python - Trabalho 2

## *Imports* estáticos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import MinMaxScaler

## Declaração de variáveis repeditas

In [ ]:
random_state = 42
alpha = 0.05
test_size = 0.3

## 4.1  Análise Exploratória de Dados
### 4.1.1   Leitura do ficheiro

In [ ]:
dados = pd.read_csv('dados/AIRPOL_data.csv', delimiter=";", header=0, decimal=',')
dados = dados.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15']).rename(columns={"Value":"Premature_Deaths","Outcome":"Disease"})
#dados = dados[dados.Country != 'All Countries']
#dados.sort_values(by=['Value'])

#### Sumário dos dados

In [ ]:
print(dados.info())

### 4.1.2   Exploração com gráficos

In [ ]:
dados.describe()

In [ ]:
features = list(dados.columns[0:8])
print(features)

In [ ]:
numericFeatures = features[4:]
numericFeatures

#### Análise Univariável

In [ ]:
# Aplicar log1p (log(1 + x)) para reduzir o impacto de valores extremos
dados_log = dados[numericFeatures].apply(lambda x: np.log1p(x))

# Aplicar MinMaxScaler
scaler = MinMaxScaler()
dados_scaled = pd.DataFrame(scaler.fit_transform(dados_log), columns=numericFeatures)

num_features = len(numericFeatures)
fig, axes = plt.subplots(nrows=num_features, ncols=2, figsize=(12, 3 + num_features))
fig.suptitle("Histogramas e Diagramas de caixa para as colunas normalizadas", fontsize=16)

for i, column in enumerate(numericFeatures):

    sns.histplot(dados_scaled[column], ax=axes[i, 0], kde=True, bins=50)
    axes[i, 0].set_title(f"Histograma de {column}")
    axes[i, 0].set_xlabel(column)
    axes[i, 0].set_ylabel("Frequência")
    axes[i, 0].tick_params(axis='x', labelrotation=45)
    axes[i, 0].xaxis.set_major_locator(plt.MaxNLocator(10))

    sns.boxplot(y=dados_scaled[column], ax=axes[i, 1])
    axes[i, 1].set_title(f"Diagrama de caixa de {column}")
    axes[i, 1].set_xlabel(column)
    axes[i, 1].set_ylabel("")
    axes[i, 1].tick_params(axis='y', labelsize=5)
    axes[i, 1].yaxis.set_major_locator(plt.MaxNLocator(10))

plt.tight_layout(rect=(0, 0, 1, 0.96))
plt.show()

#### Análise Bivariável

In [ ]:
goalAttrib = 'Premature_Deaths'

plt.figure(figsize=(15,15))
numericFeaturesWOValue = [f for f in numericFeatures if f != goalAttrib]
for i, feature in enumerate(numericFeaturesWOValue):
    rows = (len(numericFeaturesWOValue) + 1) # Adjust rows to handle odd number of features
    plt.subplot(rows,2,i+1)
    sns.scatterplot(x=dados_log[feature], y=dados_log[goalAttrib], alpha=0.6)
    plt.title("Gráfico de dispersão de " + feature + " vs " + goalAttrib)
    plt.xlabel(feature)
    plt.ylabel(goalAttrib)
    plt.grid()
plt.tight_layout()

### 4.1.3. Pré-processamento dos dados

In [ ]:
# Remover outliers
print(f"Número de linhas antes da remoção de outliers: {len(dados)}")

for col in numericFeatures:
    Q1 = dados[col].quantile(0.25)
    Q3 = dados[col].quantile(0.75)
    IQR = Q3 - Q1
    filtro = (dados[col] >= (Q1 - 1.5 * IQR)) & (dados[col] <= (Q3 + 1.5 * IQR))
    dados = dados[filtro]

print(f"Número de linhas após remoção de outliers: {len(dados)}")

### 4.1.4. Agrupamento dos dados em zonas

In [ ]:
westEuDados = dados[dados['Country'].isin(['Austria', 'Belgium', 'France', 'Germany', 'Netherlands', 'Switzerland'])]
eastEuDados = dados[dados['Country'].isin(['Poland', 'Czechia', 'Hungary'])]
soutEuDados = dados[dados['Country'].isin(['Greece', 'Spain', 'Italy', 'Portugal'])]
nortEuDados = dados[dados['Country'].isin(['Sweden', 'Denmark', 'Northern Europe', 'Finland'])]

### 4.2.1 Criar um diagrama de correlação entre a variável Premature_Deaths e os restantes atributos e interprete

# Filtração dos dados por dados de sul da Europa.

In [ ]:


soutEuDados = dados[dados['Country'].isin(['Greece', 'Spain', 'Italy', 'Portugal'])]
soutEuDadosNumericos = soutEuDados.select_dtypes(include=[np.number])


# Escolha de Representação de Correlação dos Dados: Heatmap a representar a matriz de confusão

In [ ]:
corr = soutEuDadosNumericos.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Correlação entre variáveis numéricas')
plt.tight_layout()
plt.show()

## Pequena Interpretação dos Dados

- As variáveis **Affected_Population** e **Populated_Area[km2]** apresentam uma correlação **moderadamente positiva** com o número de mortes prematuras (`Value`), com os seguintes coeficientes de correlação: **r₁ = 0.56** e **r₂ = 0.53**.

- Desta forma, podemos concluir que a maior parte das mortes foi causada por doenças que afetam populações em **áreas mais populosas e com maior densidade populacional**.

- Adicionalmente, verificamos que a variável **Air_Pollution_Average[ug/m3]** apresenta uma correlação praticamente nula com o número de mortes (**r = -0.014951**), o que indica que **a média da poluição atmosférica não tem influência significativa sobre a variável `Value`** neste conjunto de dados.


### 4.2.2 - Com o método k-fold cross validation obter um modelo de regressão linear simples para a variável Premature_Deaths usando a variável Affected_Population dos países do Southern Europe

In [ ]:
goalAttr = soutEuDados.columns[7]
feature = soutEuDados.columns[4]

print(f"Goal attribute: {goalAttr}")
print(f"Feature: {feature}")

#### Divisão dos dados (Holdout: 70% Train / 30% Test)

In [ ]:
from sklearn.model_selection import train_test_split

y = soutEuDados[goalAttr]
X = soutEuDados[[feature]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

print('Train set size: ' + str(X_train.shape) + "x_test_size" + str(X_test.shape))
print('Test set size: ' + str(y_train.shape) + "y_test_size" + str(y_test.shape))

### Modelo de Regressão Linear Simples

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train, y_train)

print(f"Função linear média: {goalAttr} = {np.round(lr.coef_, 8)} * {feature} + {np.round(lr.intercept_, 3)}")

#### Visualização da reta e da dispersão de dados de teste

In [ ]:
y_pred_test = lr.predict(X_test)

plt.figure(figsize=(8, 6))
plt.scatter(X_test, y_test, color='blue', label='Dados de Teste')
plt.plot(X_test, y_pred_test, color='red', linewidth=2, label='Reta de Regressão')
plt.xlabel('Affected_Population')
plt.ylabel('Premature_Deaths')
plt.title('Regressão Linear Simples')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

O gráfico apresentado mostra a relação entre a variável "Affected_Population" (população afetada) e "Premature_Deaths" (mortes prematuras) para os países do Sul da Europa, juntamente com a reta de regressão linear ajustada ao conjunto de teste.

Podemos observar que existe uma tendência positiva: à medida que a população afetada aumenta, o número de mortes prematuras também tende a aumentar. A reta de regressão representa a melhor aproximação linear dessa relação, de acordo com o modelo treinado.

No entanto, nota-se alguma dispersão dos pontos em torno da reta, indicando que outros fatores além da população afetada podem influenciar o número de mortes prematuras. Ainda assim, a reta ajustada sugere que a variável "Affected_Population" é um preditor relevante para "Premature_Deaths" neste contexto. 

A análise quantitativa dos erros (MAE e RMSE) nas próximas células ajudará a avaliar a precisão do modelo.

#### Calcular o MAE e o RMSE com o K-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

K = 10  # Número de folds
kFolds = KFold(n_splits=K, shuffle=True, random_state=42)

# MAE
mae_scores = cross_val_score(lr, X_train, y_train, cv=kFolds, scoring='neg_mean_absolute_error')
mae_mean = -np.mean(mae_scores)
mae_std = np.std(mae_scores)

# RMSE
mse_scores = cross_val_score(lr, X_train, y_train, cv=kFolds, scoring='neg_mean_squared_error')
rmse_scores = (-mse_scores) ** 0.5
rmse_mean = np.mean(rmse_scores)
rmse_std = np.std(rmse_scores)

print(f"MAE: {np.round(mae_mean, 3)} ± {np.round(mae_std, 3)}")
print(f"RMSE: {np.round(rmse_mean, 3)} ± {np.round(rmse_std, 3)}")

Os valores de MAE e RMSE apresentados indicam o desempenho do modelo de regressão linear simples ao prever "Premature_Deaths" com base em "Affected_Population" para os países do Sul da Europa. Se ambos os valores forem baixos e próximos entre si, isso sugere que o modelo apresenta boa precisão e que os erros de previsão não são significativamente influenciados por grandes outliers. Por outro lado, se o RMSE for consideravelmente maior que o MAE, isso indica que existem alguns pontos com erros elevados, o que pode ser resultado de dispersão ou outliers nos dados. De modo geral, quanto menores forem esses valores, melhor será o ajuste do modelo aos dados observados, reforçando a relação linear identificada no gráfico.


### 4.2.3 K-Fold cross validation
#### Otimização dos parâmetros da Árvore de regressão e visualização

In [ ]:
goalAttrib = 'Premature_Deaths'
features = list(dados.columns[0:8])
numericFeatures = features[4:]
scaler = StandardScaler()

X = soutEuDados[numericFeatures].drop(columns=[goalAttrib])
y = soutEuDados[goalAttrib]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size,random_state=random_state)

print("Stratified division of goal attribute:")
print(y_train.value_counts(normalize=True).mul(100).round(1).astype(str)+'%')
print(y_test.value_counts(normalize=True).mul(100).round(1).astype(str)+'%')

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
results = []
for max_depth in range(2, 11):
    for min_samples_split in [2, 5, 10]:
        for min_samples_leaf in [1, 2, 4]:
            dt = DecisionTreeRegressor(
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                random_state=random_state
            )
            dt.fit(X_train, y_train)
            y_pred = dt.predict(X_test)

            result = {
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'mse': mean_squared_error(y_test, y_pred),
                'r2': r2_score(y_test, y_pred)
            }
            results.append(result)

resdf = pd.DataFrame(results)
resdf = resdf.sort_values(by='r2', ascending=False)
resdf

In [ ]:
best_tree = DecisionTreeRegressor(
    max_depth=9,
    min_samples_split=5,
    min_samples_leaf=1,
    random_state=random_state
)
best_tree.fit(X_train, y_train)

plt.figure(figsize=(20, 10))
tree.plot_tree(
    best_tree,
    filled=True,
    feature_names=X_train.columns,
    rounded=True,
    fontsize=9
)
plt.title("Árvore de Decisão Otimizada")
plt.show()

#### Otimização do kernel SVM

In [ ]:
kernels = ['linear', 'rbf', 'poly']
results = []

for kernel in kernels:
    # initialize and train SVM
    svm_model = SVR(kernel=kernel)
    rmse_scores = cross_val_score(svm_model, X_train, y_train, scoring='neg_mean_squared_error', cv=20)
    # store results
    results.append({
        "model": kernel,
        "rmse": np.mean(rmse_scores),
        "std": np.std(rmse_scores)
    })

resdf = pd.DataFrame(results)
resdf = resdf.sort_values(by='rmse', ascending=True)
resdf

Com isto, vemos que o melhor kernel é o "rbf"
#### Otimização da configuração da rede neuronal

In [ ]:
def regressmodelevaluation(name_model, y_test, y_pred):
    mae = round(mean_absolute_error(y_test, y_pred), 3)
    mse = round(mean_squared_error(y_test, y_pred), 3)
    rmse = round(np.sqrt(mse), 3)
    r2 = round(r2_score(y_test, y_pred), 3)
    mape = round(mean_absolute_percentage_error(y_test, y_pred), 3)
    return pd.Series({'Model': name_model, 'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2, 'mape': mape})

In [ ]:
results = []
ncols = X_train.shape[1]

for solv in ['lbfgs', 'sgd', 'adam']:
    for nodes in range(2,ncols):
        mlp = MLPRegressor(hidden_layer_sizes=nodes,
                           activation='tanh',
                           solver=solv,
                           max_iter=1000,
                           learning_rate='adaptive',
                           early_stopping=True,
                           random_state=random_state)
        mlp.fit(X_train, y_train)

        y_pred = mlp.predict(X_test)
        result = regressmodelevaluation(f'{solv} {nodes}', y_test, y_pred)
        results.append(result)
    
resdf = pd.DataFrame(results)
resdf

In [ ]:
results = []
for solv in ['lbfgs', 'sgd', 'adam']:
    for nodes1 in range(2,6):
        for nodes2 in range(3,7):
            mlp = MLPRegressor(hidden_layer_sizes=(nodes1, nodes2),
                        activation='tanh',
                        solver=solv,
                        max_iter=1000,
                        learning_rate='adaptive',
                        early_stopping=True,
                        random_state=random_state)
            mlp.fit(X_train, y_train)

            y_pred = mlp.predict(X_test)
            result = regressmodelevaluation(f'{solv} {nodes1} {nodes2}', y_test, y_pred)
            results.append(result)
    
resdf = pd.DataFrame(results)
resdf.sort_values(by='r2', ascending=False)

Com isto, vemos que o melhor modelo é o lbfgs 4 5
#### k-fold cross validation

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)

models = []
models.append(('rgr', LinearRegression()))
models.append(('dtr', DecisionTreeRegressor(
        max_depth=9,
        min_samples_split=5,
        min_samples_leaf=1,
        random_state=random_state
    )))
models.append(('net', MLPRegressor(hidden_layer_sizes=(4, 5),
                        activation='tanh',
                        solver='lbfgs',
                        max_iter=1000,
                        learning_rate='adaptive',
                        early_stopping=True,
                        random_state=random_state)))
models.append(('svm', make_pipeline(StandardScaler(),SVR(kernel='rbf'))))

lstresults = []

for name, model in models:
    # RMSE
    mse_scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    rmse_scores = (-mse_scores) ** 0.5

    # MAE
    mae_scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
    mae_scores = -mae_scores

    # R2
    r2_scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='r2')

    lstresults.append(pd.Series({
        'model': name,
        'mean_RMSE': round(np.mean(rmse_scores), 3),
        'std_RMSE': round(np.std(rmse_scores), 3),
        'mean_MAE': round(np.mean(mae_scores), 3),
        'std_MAE': round(np.std(mae_scores), 3),
        'mean_R2': round(np.mean(r2_scores), 3),
        'std_R2': round(np.std(r2_scores), 3)
    }))
    print(f"Model {name} done")

resdf = pd.DataFrame(lstresults)
resdf.sort_values(by='mean_R2', ascending=False)

### 4.2.4 Comparar os resultados obtidos pelos modelos referidos, usando o erro médio absoluto (MAE) e a raiz quadrada do erro médio (RMSE).

Analisando o erro médio absoluto (MAE) dos modelos avaliados na tabela acima, observa-se que a SVM (375.694) apresenta o menor valor médio, seguida pela árvore de regressão (dtr, 400.754), regressão linear (rgr, 418.009) e, por fim, a rede neural (net, 594.638).

Quanto à raiz quadrada do erro médio (RMSE), a árvore de regressão (dtr, 1755.514) apresenta o menor valor, seguida pela regressão linear (rgr, 1768.854), SVM (2068.130) e rede neural (net, 2070.886).


### 4.2.5 Justificar se os resultados obtidos para os dois melhores modelos são estatisticamente significativos, e indicar o(s) modelo(s) com melhor desempenho

Os dois melhores modelos são o da regressão linear múltipla e o da árvore de regressão

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)

models = []
models.append(('rgr', LinearRegression()))
models.append(('dtr', DecisionTreeRegressor(
        max_depth=9,
        min_samples_split=5,
        min_samples_leaf=1,
        random_state=random_state
    )))

lstresults = []
rgr_results_rmse = []
dtr_results_rmse = []

for name, model in models:
    # RMSE
    mse_scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    rmse_scores = (-mse_scores) ** 0.5

    # MAE
    mae_scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
    mae_scores = -mae_scores

    if name == 'rgr':
        rgr_results_rmse = rmse_scores
        rgr_results_mae = mae_scores
    else:
        dtr_results_rmse = rmse_scores
        dtr_results_mae = mae_scores

In [ ]:
from scipy.stats import shapiro, ttest_ind, mannwhitneyu

t_stat_mae_rgr, p_value_mae_rgr = shapiro(rgr_results_mae)
t_stat_mae_dtr, p_value_mae_dtr = shapiro(dtr_results_mae)
print(f"Shapiro-Wilk MAE RGR: p-value={p_value_mae_rgr}")
print(f"Shapiro-Wilk MAE DTR: p-value={p_value_mae_dtr}")

if p_value_mae_rgr > alpha and p_value_mae_dtr > alpha:
    print("MAE de RGR e DTR seguem distribuição normal.")
    t_stat, p_val_t = ttest_ind(rgr_results_mae, dtr_results_mae, equal_var=False)
    print(f"Teste t de Student (MAE): p-value={p_val_t}")
    if p_val_t < alpha:
        print("MAE de RGR e DTR são significativamente diferentes.")
    else:
        print("MAE de RGR e DTR não são significativamente diferentes.")
else:
    print("MAE de RGR e DTR não seguem distribuição normal.")
    u_stat, p_val_u = mannwhitneyu(rgr_results_mae, dtr_results_mae)
    print(f"Teste U de Mann-Whitney (MAE): p-value={p_val_u}")
    if p_val_u < alpha:
        print("MAE de RGR e DTR são significativamente diferentes.")
    else:
        print("MAE de RGR e DTR não são significativamente diferentes.")

print("\n---\n")

t_stat_rmse_rgr, p_value_rmse_rgr = shapiro(rgr_results_rmse)
t_stat_rmse_dtr, p_value_rmse_dtr = shapiro(dtr_results_rmse)
print(f"Shapiro-Wilk RMSE RGR: p-value={p_value_rmse_rgr}")
print(f"Shapiro-Wilk RMSE DTR: p-value={p_value_rmse_dtr}")

if p_value_rmse_rgr > alpha and p_value_rmse_dtr > alpha:
    print("RMSE de RGR e DTR seguem distribuição normal.")
    t_stat, p_val_t = ttest_ind(rgr_results_rmse, dtr_results_rmse, equal_var=False)
    print(f"Teste t de Student (RMSE): p-value={p_val_t}")
    if p_val_t < alpha:
        print("RMSE de RGR e DTR são significativamente diferentes.")
    else:
        print("RMSE de RGR e DTR não são significativamente diferentes.")
else:
    print("RMSE de RGR e DTR não seguem distribuição normal.")
    u_stat, p_val_u = mannwhitneyu(rgr_results_rmse, dtr_results_rmse)
    print(f"Teste U de Mann-Whitney (RMSE): p-value={p_val_u}")
    if p_val_u < alpha:
        print("RMSE de RGR e DTR são significativamente diferentes.")
    else:
        print("RMSE de RGR e DTR não são significativamente diferentes.")

Nos dois modelos e com os valores, não apresentam-se diferenças significativas. Então os melhores modelos são o da árvore de regressão e o da regressão linear múltipla.

### 4.3.1 Novo atributo "RespDisease"

In [ ]:
dados4regioes = pd.concat([westEuDados, eastEuDados, soutEuDados, nortEuDados], ignore_index=False)
dados4regioes

Este código foi inspirado por esta [thread](https://stackoverflow.com/questions/67420811/a-new-column-in-pandas-which-value-depends-on-other-columns) no stack**overflow**

In [ ]:
goalAttrib = 'RespDisease'

def calc_resp_disease(row):
    respDiseases = ['Asthma', 'Chronic obstructive pulmonary disease']
    if row['Disease'] in respDiseases:
        return 1
    else:
        return 0

dados4regioes[goalAttrib] = dados4regioes.apply(calc_resp_disease, axis=1)

dados4regioes

### 4.3.2 K-Fold cross validation

#### Preparação dos valores

In [ ]:
features = list(dados4regioes.columns[0:9])
numericFeatures = features[4:]
print(numericFeatures)

X = dados4regioes[numericFeatures].drop(columns=[goalAttrib])
y = dados4regioes[goalAttrib]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size,random_state=random_state)

print("Stratified division of goal attribute:")
print(y_train.value_counts(normalize=True).mul(100).round(1).astype(str)+'%')
print(y_test.value_counts(normalize=True).mul(100).round(1).astype(str)+'%')

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Otimização dos parâmetros da Árvore de regressão

In [ ]:
results = []
for max_depth in range(2, 11):
    for min_samples_split in [2, 5, 10]:
        for min_samples_leaf in [1, 2, 4]:
            dt = DecisionTreeRegressor(
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                random_state=random_state
            )
            dt.fit(X_train_scaled, y_train)
            y_pred = dt.predict(X_test_scaled)

            result = {
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'mse': mean_squared_error(y_test, y_pred),
                'r2': r2_score(y_test, y_pred)
            }
            results.append(result)

resdf = pd.DataFrame(results)
resdf = resdf.sort_values(by='r2', ascending=False)
resdf

Assim, vemos que a melhor árvore tem os seguintes parâmetros:
* max_depth = 10
* min_samples_split = 2
* min_samples_leaf = 2
#### Otimização do kernel SVM

In [ ]:
kernels = ['linear', 'rbf', #'poly'
           ]
results = []

for kernel in kernels:
    # initialize and train SVM
    svm_model = SVR(kernel=kernel)
    rmse_scores = cross_val_score(svm_model, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=10)
    # store results
    results.append({
        "model": kernel,
        "rmse": np.mean(rmse_scores),
        "std": np.std(rmse_scores)
    })
    print(f"{kernel} done")

resdf = pd.DataFrame(results)
resdf = resdf.sort_values(by='rmse', ascending=True)
resdf

Comparando os valores, chegamos à conclusão que o melhor kernel é o "linear".
(O "poly" não foi mostrado visto que demora demasiado (mais de meia hora com cv=2))

#### Otimização da configuração da rede neuronal

In [ ]:
def regressmodelevaluation(name_model, y_test, y_pred):
    mae = round(mean_absolute_error(y_test, y_pred), 3)
    mse = round(mean_squared_error(y_test, y_pred), 3)
    rmse = round(np.sqrt(mse), 3)
    r2 = round(r2_score(y_test, y_pred), 3)
    mape = round(mean_absolute_percentage_error(y_test, y_pred), 3)
    return pd.Series({'Model': name_model, 'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2, 'mape': mape})

In [ ]:
results = []
for solv in ['lbfgs', 'sgd', 'adam']:
    for nodes1 in range(2,6):
        for nodes2 in range(3,7):
            mlp = MLPRegressor(hidden_layer_sizes=(nodes1, nodes2),
                        activation='tanh',
                        solver=solv,
                        max_iter=1000,
                        learning_rate='adaptive',
                        early_stopping=True,
                        random_state=random_state)
            mlp.fit(X_train_scaled, y_train)

            y_pred = mlp.predict(X_test_scaled)
            result = regressmodelevaluation(f'{solv} {nodes1} {nodes2}', y_test, y_pred)
            results.append(result)
    
resdf = pd.DataFrame(results)
resdf.sort_values(by='r2', ascending=False)

Com isto, vemos que o melhor modelo é o lbfgs 5 6
#### Otimização da configuração dos K-vizinhos mais próximos

In [ ]:
num_holdouts = 20

results = []

for k in range(1, 51, 2):
    rmse_scores = []
    for _ in range(num_holdouts):
        Xhd_train, Xhd_test, yhd_train, yhd_test = train_test_split(X_train_scaled, y_train, test_size=test_size)

        knn_model = KNeighborsRegressor(n_neighbors=k)
        knn_model.fit(Xhd_train, yhd_train)

        y_pred = knn_model.predict(Xhd_test)

        mse = mean_squared_error(yhd_test, y_pred)
        rmse_scores.append(np.sqrt(mse))

    results.append({
        "k": k,
        "rmse_mean": np.mean(rmse_scores),
        "rmse_std": np.std(rmse_scores)
    })

resdf = pd.DataFrame(results)
resdf = resdf.sort_values(by='rmse_mean', ascending=True)
resdf

k = 15 é o ideal aqui (?)

#### k-fold cross validation

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)

models = []
#models.append(('rgr', LinearRegression()))
models.append(('dtr', DecisionTreeRegressor(
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=2,
        random_state=random_state
    )))
models.append(('net', MLPRegressor(hidden_layer_sizes=(5, 6),
                        activation='tanh',
                        solver='lbfgs',
                        max_iter=1000,
                        learning_rate='adaptive',
                        early_stopping=True,
                        random_state=random_state)))
models.append(('knn', KNeighborsRegressor(n_neighbors=21)))
models.append(('svm', make_pipeline(StandardScaler(),SVR(kernel='linear'))))

lstresults = []

for name, model in models:
    # RMSE
    mse_scores = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
    rmse_scores = (-mse_scores) ** 0.5

    # MAE
    mae_scores = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error')
    mae_scores = -mae_scores

    # R2
    r2_scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')

    lstresults.append(pd.Series({
        'model': name,
        'mean_RMSE': round(np.mean(rmse_scores), 3),
        'std_RMSE': round(np.std(rmse_scores), 3),
        'mean_MAE': round(np.mean(mae_scores), 3),
        'std_MAE': round(np.std(mae_scores), 3),
        'mean_R2': round(np.mean(r2_scores), 3),
        'std_R2': round(np.std(r2_scores), 3)
    }))
    print(f"Model {name} done")

resdf = pd.DataFrame(lstresults)
resdf.sort_values(by='mean_R2', ascending=False)